# ATHENA ZAPANTIS

In [1]:
# This is not generalizable to the Marko truss :(
# This really only works with a triangle
# My goal is to make it generalizable because this relies on
# being in 2-D and having 3 nodes

In [2]:
import math as m
import numpy as np

class Truss:
    
    def __init__(self, elements = [], fixed_DOFs = [3,4,5,6], external_forces = [100000, 0, 0, 0, 0, 0], imposed_displacements = [0, 0, 0, 0, 0, 0]):
        self.elements = elements
        self.fixed_DOFs = fixed_DOFs
        self.external_forces = np.array(external_forces)
        self.imposed_displacements = np.array(imposed_displacements)
        self.nodes = set()
        
        for element in self.elements:
            self.nodes.add(tuple(element.node_i))
            self.nodes.add(tuple(element.node_j)) 
    
    def define_material_properties(self, material_properties):
        self.material_properties =  material_properties
        
    def define_section_properties(self, section_properties):
        self.section_properties = section_properties
        
    def define_fixed_DOFs(self, s):
        self.fixed_DOFs = s
        
    def add_element(self, element):
        '''accepts entries of the class Element'''
        self.elements.append(element)
        
    def get_material_properties(self):
        return self.material_properties
    
    def get_section_properties(self):
        return self.section_properties
    
    def get_nodes(self):
        print(self.nodes)
        
    def define_global_dof_for_element(self, element):
        node_i_index = list(self.nodes).index(tuple(element.node_i))  
        node_j_index = list(self.nodes).index(tuple(element.node_j))  

        dof_i = [node_i_index * 2, node_i_index * 2 + 1]  # Node i's DOF (x and y)
        dof_j = [node_j_index * 2, node_j_index * 2 + 1]  # Node j's DOF (x and y)
    
        return dof_i + dof_j  
    
    def calculate_big_mac(self):
        '''calculates the total global stiffness matrix'''
        np.set_printoptions(precision=0)
        # global DOFs for each element
        element_global_dof = [self.define_global_dof_for_element(self.elements[i]) for i in range(len(self.elements))]
        num_dof = len(self.nodes) * 2

        k_global = np.zeros((num_dof, num_dof))

        ke_globals = [element.calculate_element_global_stiffness_matrix() for element in self.elements]

        # ASSEMBLE THE PATTY, LETTUCE, TOMATO IN THEIR PLACES!!
        for i in range(len(ke_globals)):
            dof = element_global_dof[i]
            for m in range(4):
                for n in range(4):
                    k_global[dof[m], dof[n]] += ke_globals[i][m, n]
        return k_global
    
    def calculate_element_internal_forces(self):
        num_dof = len(self.nodes) * 2
        u = np.zeros((num_dof,1)) # initialize displacement vector
        element_global_dof = [self.define_global_dof_for_element(self.elements[i]) for i in range(len(self.elements))]
        u_element_global = [u[element_global_dof[i]] for i in range(len(element_global_dof))]
        ke_locals = [element.calculate_element_local_stiffness_matrix() for element in self.elements]
        
        theta = [self.elements[i].calculate_element_orientation() for i in range(len(elements))]
        beta = [self.elements[i].define_rotation_matrix(theta[i]) for i in range(len(elements))]
        delta = [beta[i] * u_element_global[i] for i in range(len(u_element_global))]
                
        P = [ke_locals[i] @ delta[i] for i in range(len(ke_locals))]

        return P
    
    def calculate_global_displacements(self):        
        num_dof = len(self.nodes) * 2
        u = np.zeros((num_dof,1)) # initialize displacement vector
        F = np.array(self.external_forces) # initialize F
        dofs = set([i + 1 for i in range(num_dof)]) #All the DOFs
        s = self.fixed_DOFs
        
        p = dofs.difference(s)
        p = sorted(list(p)) # Free DOFs
        
        self.fixed_DOFs = [e - 1 for e in self.fixed_DOFs]
        p = [e - 1 for e in p] # Make Python not annoying
        s = self.fixed_DOFs

        k_global = self.calculate_big_mac()
        
        Kpp = k_global[np.ix_(p, p)]   
        Kss = k_global[np.ix_(s, s)]   
        Kps = k_global[np.ix_(p, s)]   
        Ksp = k_global[np.ix_(s, p)]
        Fp = F[p]        
        
        us = self.imposed_displacements[self.fixed_DOFs] # Grab imposed displacements
        up = np.linalg.inv(Kpp) @ (Fp - (Kps @ us))
        u[p] = up.reshape(-1,1) # the dimensions of the vectors got a little funky
        Fs = (Ksp @ up) + (Kss @ us)
        
        return u
    
    def calculate_reaction_forces(self):
        num_dof = len(self.nodes) * 2
        u = np.zeros((num_dof,1)) # initialize displacement vector
        F = np.array(self.external_forces) # initialize F
        dofs = set([i + 1 for i in range(num_dof)]) #All the DOFs
        s = self.fixed_DOFs
        
        p = dofs.difference(s)
        p = sorted(list(p)) # Free DOFs
        
        self.fixed_DOFs = [e - 1 for e in self.fixed_DOFs]
        p = [e - 1 for e in p] # Make Python not annoying
        s = self.fixed_DOFs

        k_global = self.calculate_big_mac()
        
        Kpp = k_global[np.ix_(p, p)]   
        Kss = k_global[np.ix_(s, s)]   
        Kps = k_global[np.ix_(p, s)]   
        Ksp = k_global[np.ix_(s, p)]
        Fp = F[p]        
        
        us = self.imposed_displacements[self.fixed_DOFs] # Grab imposed displacements
        up = np.linalg.inv(Kpp) @ (Fp - (Kps @ us))
        u[p] = up.reshape(-1,1) # the dimensions of the vectors got a little funky
        
        return (Ksp @ up) + (Kss @ us)

In [3]:
class Element:
    
    def __init__(self, node_i = [0,0], node_j = [0,0], E = 0, A = 0):
        self.node_i = node_i
        self.node_j = node_j
        self.E = E
        self.A = A
        self.L = np.linalg.norm(np.array(self.node_i) - np.array(self.node_j))
    
    def calculate_element_local_stiffness_matrix(self):
        ke_local = np.zeros((4,4))
        stiffness = self.E * self.A / self.L
        ke_local[0, 0] = stiffness
        ke_local[0, 2] = -stiffness
        ke_local[2, 0] = -stiffness
        ke_local[2, 2] = stiffness
        return ke_local
    
    def define_rotation_matrix(self, theta):
        unit_rotation = np.array(
            [(m.cos(theta), m.sin(theta)), 
             (-m.sin(theta), m.cos(theta))])
        return np.vstack((
            np.hstack((unit_rotation, np.zeros((2, 2)))),
            np.hstack((np.zeros((2, 2)), unit_rotation))
        ))
    
    def calculate_element_length(self):
        self.L = np.linalg.norm(self.node_i, self.node_j)
        return self.L
    
    def calculate_element_orientation(self):
        delta = [self.node_i[0] - self.node_j[0], self.node_i[1] - self.node_j[1]]
        return m.atan2(delta[1], delta[0])
    
    def calculate_element_global_stiffness_matrix(self):
        theta = self.calculate_element_orientation()
        beta = self.define_rotation_matrix(theta)
        ke_local = self.calculate_element_local_stiffness_matrix()
        return beta.T @ ke_local @ beta

In [4]:
nodes = [[0,0], [-100,-100], [100, -100]] # define [x,y] coordinates of each node
material_properties = [29000000, 10000, 15000000] # define E of steel, wood, carbon fiber
section_properties = [1, 2, 3.6] # define cross-sectional area

element1 = Element(nodes[0], nodes[1], material_properties[0], section_properties[1])
element2 = Element(nodes[0], nodes[2], material_properties[2], section_properties[0])
element3 = Element(nodes[1], nodes[2], material_properties[1], section_properties[2])
elements = [element1, element2, element3]

my_truss = Truss(elements) # the default settings are the problem setup

k_global = my_truss.calculate_big_mac()
displacement_field = my_truss.calculate_global_displacements()
reaction_forces = my_truss.calculate_reaction_forces()
internal_forces = my_truss.calculate_element_internal_forces()

print(f'big mac: {k_global}')
print(f'displacement field: {displacement_field}')
print(f'reaction forces: {reaction_forces}')
print(f'internal forces: {internal_forces}')

big mac: [[ 2.e+05  2.e+05 -2.e+02  2.e-14 -2.e+05 -2.e+05]
 [ 2.e+05  2.e+05  2.e-14 -3.e-30 -2.e+05 -2.e+05]
 [-2.e+02  2.e-14  5.e+04 -5.e+04 -5.e+04  5.e+04]
 [ 2.e-14 -3.e-30 -5.e+04  5.e+04  5.e+04 -5.e+04]
 [-2.e+05 -2.e+05 -5.e+04  5.e+04  3.e+05  2.e+05]
 [-2.e+05 -2.e+05  5.e+04 -5.e+04  2.e+05  3.e+05]]
displacement field: [[ 556.]
 [-556.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]]
reaction forces: [  99575.   99149.  -99575. -199149.]
internal forces: [array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])]


Commentary:

I believe the big mac was not assembled correctly. I think the third element's ke_global might be wrong for some reason, but I'm not sure why because I think the first two are fine.

I am guessing that the values for the displacement field are so large because of this reason. However, it's important to note that the indeed the the first DOF should have a positive displacement and the second DOF should have a negative displacement.

In regard to the reaction forces, there may be a problem with indexing since it appears to be balancing a y applied force.

Also, the internal forces should certainly be greater than zero. I'm also not sure why it has dimension 4x4 for each element.

Some fixes are implemented below with the help of ChatGPT. 

Overall, I think the general approach is there but it needs some debugging.

In [5]:
import math as m
import numpy as np

class Truss:
    
    def __init__(self, elements = [], fixed_DOFs = [3,4,5,6], external_forces = [100000, 0, 0, 0, 0, 0], imposed_displacements = [0, 0, 0, 0, 0, 0]):
        self.elements = elements
        self.fixed_DOFs = fixed_DOFs
        self.external_forces = np.array(external_forces)
        self.imposed_displacements = np.array(imposed_displacements)
        self.nodes = set()
        
        for element in self.elements:
            self.nodes.add(tuple(element.node_i))
            self.nodes.add(tuple(element.node_j)) 
    
    def define_material_properties(self, material_properties):
        self.material_properties =  material_properties
        
    def define_section_properties(self, section_properties):
        self.section_properties = section_properties
        
    def define_fixed_DOFs(self, s):
        self.fixed_DOFs = s
        
    def add_element(self, element):
        '''accepts entries of the class Element'''
        self.elements.append(element)
        
    def get_material_properties(self):
        return self.material_properties
    
    def get_section_properties(self):
        return self.section_properties
    
    def get_nodes(self):
        print(self.nodes)
        
    def define_global_dof_for_element(self, element):
        node_i_index = list(self.nodes).index(tuple(element.node_i))  
        node_j_index = list(self.nodes).index(tuple(element.node_j))  

        dof_i = [node_i_index * 2, node_i_index * 2 + 1]  # Node i's DOF (x and y)
        dof_j = [node_j_index * 2, node_j_index * 2 + 1]  # Node j's DOF (x and y)
    
        return dof_i + dof_j  
    
    def calculate_big_mac(self):
        '''calculates the total global stiffness matrix'''
        np.set_printoptions(precision=0)
        # global DOFs for each element
        element_global_dof = [self.define_global_dof_for_element(self.elements[i]) for i in range(len(self.elements))]
        num_dof = len(self.nodes) * 2

        k_global = np.zeros((num_dof, num_dof))

        ke_globals = [element.calculate_element_global_stiffness_matrix() for element in self.elements]

        # ASSEMBLE THE PATTY, LETTUCE, TOMATO IN THEIR PLACES!!
        for i in range(len(ke_globals)):
            dof = element_global_dof[i]
            for m in range(4):
                for n in range(4):
                    k_global[dof[m], dof[n]] += ke_globals[i][m, n]
        return k_global
    
    def calculate_element_internal_forces(self):
        num_dof = len(self.nodes) * 2
        u = np.zeros((num_dof,1)) # initialize displacement vector
        element_global_dof = [self.define_global_dof_for_element(self.elements[i]) for i in range(len(self.elements))]
        u_element_global = [u[element_global_dof[i]] for i in range(len(element_global_dof))]
        ke_locals = [element.calculate_element_local_stiffness_matrix() for element in self.elements]
        
        theta = [self.elements[i].calculate_element_orientation() for i in range(len(self.elements))]
        beta = [self.elements[i].define_rotation_matrix(theta[i]) for i in range(len(self.elements))]
        delta = [beta[i] @ u_element_global[i] for i in range(len(u_element_global))]
                
        P = [ke_locals[i] @ delta[i] for i in range(len(ke_locals))]

        return P
    
    def calculate_global_displacements(self):        
        num_dof = len(self.nodes) * 2
        u = np.zeros((num_dof,1)) # initialize displacement vector
        F = np.array(self.external_forces) # initialize F
        dofs = set([i + 1 for i in range(num_dof)]) # All the DOFs
        s = self.fixed_DOFs
        
        p = dofs.difference(s)
        p = sorted(list(p)) # Free DOFs
        
        self.fixed_DOFs = [e - 1 for e in self.fixed_DOFs]
        p = [e - 1 for e in p] # Make Python not annoying
        s = self.fixed_DOFs

        k_global = self.calculate_big_mac()
        
        Kpp = k_global[np.ix_(p, p)]   
        Kss = k_global[np.ix_(s, s)]   
        Kps = k_global[np.ix_(p, s)]   
        Ksp = k_global[np.ix_(s, p)]
        Fp = F[p]        
        
        us = self.imposed_displacements[s] # Grab imposed displacements
        up = np.linalg.inv(Kpp) @ (Fp - (Kps @ us))
        u[p] = up.reshape(-1,1) # the dimensions of the vectors got a little funky
        Fs = (Ksp @ up) + (Kss @ us)
        
        return u
    
    def calculate_reaction_forces(self):
        num_dof = len(self.nodes) * 2
        u = np.zeros((num_dof,1)) # initialize displacement vector
        F = np.array(self.external_forces) # initialize F
        dofs = set([i + 1 for i in range(num_dof)]) # All the DOFs
        s = self.fixed_DOFs
        
        p = dofs.difference(s)
        p = sorted(list(p)) # Free DOFs
        
        self.fixed_DOFs = [e - 1 for e in self.fixed_DOFs]
        p = [e - 1 for e in p] # Make Python not annoying
        s = self.fixed_DOFs

        k_global = self.calculate_big_mac()
        
        Kpp = k_global[np.ix_(p, p)]   
        Kss = k_global[np.ix_(s, s)]   
        Kps = k_global[np.ix_(p, s)]   
        Ksp = k_global[np.ix_(s, p)]
        Fp = F[p]        
        
        us = self.imposed_displacements[s] # Grab imposed displacements
        up = np.linalg.inv(Kpp) @ (Fp - (Kps @ us))
        u[p] = up.reshape(-1,1) # the dimensions of the vectors got a little funky
        
        return (Ksp @ up) + (Kss @ us)

class Element:
    
    def __init__(self, node_i = [0,0], node_j = [0,0], E = 0, A = 0):
        self.node_i = node_i
        self.node_j = node_j
        self.E = E
        self.A = A
        self.L = np.linalg.norm(np.array(self.node_i) - np.array(self.node_j))
    
    def calculate_element_local_stiffness_matrix(self):
        ke_local = np.zeros((4,4))
        stiffness = self.E * self.A / self.L
        ke_local[0, 0] = stiffness
        ke_local[0, 2] = -stiffness
        ke_local[2, 0] = -stiffness
        ke_local[2, 2] = stiffness
        return ke_local
    
    def define_rotation_matrix(self, theta):
        unit_rotation = np.array(
            [(m.cos(theta), m.sin(theta)), 
             (-m.sin(theta), m.cos(theta))])
        return np.vstack((
            np.hstack((unit_rotation, np.zeros((2, 2)))),
            np.hstack((np.zeros((2, 2)), unit_rotation))
        ))
    
    def calculate_element_length(self):
        self.L = np.linalg.norm(np.array(self.node_i) - np.array(self.node_j))
        return self.L
    
    def calculate_element_orientation(self):
        delta = [self.node_i[0] - self.node_j[0], self.node_i[1] - self.node_j[1]]
        return m.atan2(delta[1], delta[0])
    
    def calculate_element_global_stiffness_matrix(self):
        theta = self.calculate_element_orientation()
        beta = self.define_rotation_matrix(theta)
        ke_local = self.calculate_element_local_stiffness_matrix()
        return beta.T @ ke_local @ beta

# Define the nodes and elements
nodes = [[0,0], [-100,-100], [100, -100]] # define [x,y] coordinates of each node
material_properties = [29000000, 10000, 15000000] # define E of steel, wood, carbon fiber
section_properties = [1, 2, 3.6] # define cross-sectional area

element1 = Element(nodes[0], nodes[1], material_properties[0], section_properties[1])
element2 = Element(nodes[0], nodes[2], material_properties[2], section_properties[0])
element3 = Element(nodes[1], nodes[2], material_properties[1], section_properties[2])
elements = [element1, element2, element3]

# Define the truss structure
my_truss = Truss(elements) # the default settings are the problem setup

# Calculate global stiffness matrix and displacements
k_global = my_truss.calculate_big_mac()
displacement_field = my_truss.calculate_global_displacements()
reaction_forces = my_truss.calculate_reaction_forces()
internal_forces = my_truss.calculate_element_internal_forces()

print(f'big mac: {k_global}')
print(f'displacement field: {displacement_field}')
print(f'reaction forces: {reaction_forces}')
print(f'internal forces: {internal_forces}')


big mac: [[ 2.e+05  2.e+05 -2.e+02  2.e-14 -2.e+05 -2.e+05]
 [ 2.e+05  2.e+05  2.e-14 -3.e-30 -2.e+05 -2.e+05]
 [-2.e+02  2.e-14  5.e+04 -5.e+04 -5.e+04  5.e+04]
 [ 2.e-14 -3.e-30 -5.e+04  5.e+04  5.e+04 -5.e+04]
 [-2.e+05 -2.e+05 -5.e+04  5.e+04  3.e+05  2.e+05]
 [-2.e+05 -2.e+05  5.e+04 -5.e+04  2.e+05  3.e+05]]
displacement field: [[ 556.]
 [-556.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]]
reaction forces: [  99575.   99149.  -99575. -199149.]
internal forces: [array([[0.],
       [0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.],
       [0.]])]
